## TLDR/ Read.me For Features

Before beginning the task I needed to work on data in order to generate features for my model. I have followed below steps in order to achieve my goal:

<ol>
<li>First I needed to join original data provided with geographical data by using join keys. </li> 
<li>Secondly, I have decided to calculate features based on given rows which represents transactions.I have named this mi-phase dataframe as 'staging dataframe'</li> 
    
In the staging dataframe I have generated below features:

 <b>vac_duration</b>= Indicates the duration of the booked/searched vacation in days.<br>
 <b>book_vac_duration</b>=The lag between booking/searching and the beginning  of the vacation in days<br>
 <b>book_days_ago</b>= Indicates how many days ago booking/searching is performed.<br>
 <b>child_per_adult</b>= It is basically the number of children per adult.<br>
 <b>booked</b>= Indicates if the action is booking or not. 1 indicates booking, 0 indicates search.<br>
 <b>searched</b>=Indicates if the action is searching or not. 1 indicates searching, 0 indicates book.<br>
 <b>is_summer</b>=Indicates if the vacation period is in summer or not. <br>
 <b>christmas</b>=Indicates if the vacation period is at Christmas or not. <br>
 <b>vac_distance1</b>= It represents the distance from the origin to destination in KMs. Geopy package is utilized.<br>
 <b>vac_distance2</b>= It represents the distance from the origin to destination in KMs. Math formula is utilized.<br>
 <b>Note:</b> I wanted to compare 2 different distance calculations.<br>
 
 <li> In the staging dataframe all features were calculated based on transaction rows. But in the model, it is a better idea to utilize user-based features since each prediction will be performed for each individual user. That's why I decided to aggregate features on the user level. I have classified some attributes to be summed and some to be averaged. Then I have basically aggregated them </li> 
 </ol>
 
 


In [1]:
%pip install -r requirements.txt

  ERROR: Could not find a version that satisfies the requirement dateutil (from -r requirements.txt (line 14)) (from versions: none)
ERROR: No matching distribution found for dateutil (from -r requirements.txt (line 14))
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import date, datetime, timedelta
import geopy.distance
import math
import holidays # I havent used this lib because of temporary errors
#I kept it just to notify you that I have thought of using it
import itertools
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from IPython.display import display
%matplotlib inline

/home/him/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [3]:
df_original=pd.read_csv('events_1_1.csv.gz', compression='gzip', header=0,    sep=',', quotechar='"', error_bad_lines=False)

In [4]:
df_original.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0


In [5]:
df_original.describe()

,num_adults,num_children
count,47007.000000,47007.000000
mean,1.676261,0.146723
std,0.988610,0.477620
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,2.000000,0.000000
max,9.000000,5.000000


In [6]:
# How many distinct events are there in the dataset and what are they?
df_original['event_type'].unique()

array(['search', 'book'], dtype=object)

In [7]:
#I see that there could be more than 1 row related to same user. 
#I need to consider this while generating user features
df_original['user_id'].nunique()

31179

In [8]:
#Data types are important to generate features
df_original.dtypes

ts              object
event_type      object
user_id         object
date_from       object
date_to         object
origin          object
destination     object
num_adults       int64
num_children     int64
dtype: object

In [9]:
iata=pd.read_csv('iata_1_1.csv', header=0,    sep=',', quotechar='"', error_bad_lines=False)

In [10]:
iata.head()

,iata_code,lat,lon
0,LON,51.50,-0.17
1,MOW,55.75,37.70
2,NYC,40.71,-74.01
3,PAR,48.85,2.35
4,IST,41.01,28.95


In [11]:
iata.describe()

,lat,lon
count,500.000000,500.00000
mean,26.468760,15.38048
std,24.869859,89.05271
min,-43.530000,-165.45000
25%,16.885000,-75.28500
50%,33.550000,13.26000
75%,43.442500,103.67500
max,64.840000,177.44000


In [12]:
iata.dtypes

iata_code     object
lat          float64
lon          float64
dtype: object

## Merged Dataframes

In [13]:
#Simply merging 2 dataframes by using the key attribute
df=df_original.merge(iata, left_on='origin', right_on='iata_code', how='left')

In [14]:
#in order to distinguish origin and destination
df.rename(columns={"lat": "origin_lat", "lon": "origin_lon"}, inplace=True)

In [15]:
# no need for this column anymore
df.drop(columns='iata_code', inplace=True)

In [16]:
#Same steps for destination
df=df.merge(iata, left_on='destination', right_on='iata_code', how='left')
df.rename(columns={"lat": "dest_lat", "lon": "dest_lon"}, inplace=True)
df.drop(columns='iata_code', inplace=True)

In [17]:
df.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,origin_lat,origin_lon,dest_lat,dest_lon
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,48.85,2.35,40.71,-74.01
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,50.12,8.68,38.90,-77.04
2,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,50.04,8.56,38.90,-77.04
3,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,52.52,13.41,50.88,7.12
4,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,52.52,13.41,50.93,6.95


## Pre-Feature Engineering Staging Dataframe

In [18]:
#To be sure that they are in the correct dattime format
df['date_from']=pd.to_datetime(df['date_from'])
df['date_to']=pd.to_datetime(df['date_to'])
df['ts']=pd.to_datetime(df['ts'])

In [19]:
#Vacation duration Feature - this feature could be significant for the model
df['vac_duration']= (df['date_to']-df['date_from']).dt.days

In [20]:
# The time lag between booking/searching and the begining of the vacation
df['book_search_duration']= (df['date_from']-df['ts']).dt.days

In [21]:
# Recency of booking or searching. How many days ago booking/searching has been performed
df['book_days_ago']=(datetime.now()-df['ts']).dt.days

In [22]:
# I am not sure this would be a significant feature 
#since it would be highly correlated with num_children and num_adults anyways I wanted to give a try
df['child_per_adult']=df['num_children']/df['num_adults']

In [23]:
# booked or searched this would be a significant feature
#did not have time to refactor
book=[]
search=[]
for event in df['event_type']:
    if event == 'book':
        booked=1
        searched=0
    else:
        booked=0
        searched=1
    book.append(booked)
    search.append(searched)
df['booked']=book
df['searched']=search

In [24]:
#is_summer I guess season summer has significant affect on booking decisions
#did not have time to refactor
summers=[]
for time in df['date_from']:
    if time.month in (6,7,8):
        summer=1
    else:
        summer=0
    summers.append(summer)
df['is_summer']=summers

In [25]:
#Christmas I guess season summer has significant affect on booking decisions
#did not have time to refactor
cdays=['1225','1226','1227','1228','1229','1230','1231','0101']
date_from = pd.to_datetime(df['date_from'], errors='coerce').dt.strftime('%m%d')
christmas = []
for md in date_from:
    if md in cdays and md is not None:
        holiday=1
    else:
        holiday=0
    christmas.append(holiday)
df['christmas']=christmas
    

In [26]:
#I wanted to check if it works
df[df["christmas"]==1].head()
#de_holidays = holidays.DE()
#de_holidays

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,origin_lat,...,dest_lat,dest_lon,vac_duration,book_search_duration,book_days_ago,child_per_adult,booked,searched,is_summer,christmas
4367,2017-04-21 05:27:57,search,ce5469,2017-12-25,2017-12-25,MSY,CHI,2,0,29.99,...,41.85,-87.65,0.0,247.0,1196,0.0,0,1,0,1
4368,2017-04-21 05:27:57,search,ce5469,2017-12-25,2017-12-25,MSY,CHI,2,0,29.95,...,41.85,-87.65,0.0,247.0,1196,0.0,0,1,0,1
6176,2017-04-23 00:03:09,search,7aff52,2017-12-28,2018-01-12,MUC,HNL,2,0,48.14,...,21.32,-157.93,15.0,248.0,1194,0.0,0,1,0,1
6177,2017-04-23 00:03:09,search,7aff52,2017-12-28,2018-01-12,MUC,HNL,2,0,48.14,...,21.32,-157.92,15.0,248.0,1194,0.0,0,1,0,1
6178,2017-04-23 00:03:09,search,7aff52,2017-12-28,2018-01-12,MUC,HNL,2,0,48.35,...,21.32,-157.93,15.0,248.0,1194,0.0,0,1,0,1


### Distance Calculation
I wanted to appyl 2 different calculations in order to compare them and I will decide later to choose one of them if I really need to choose.
<li><var>First one relies on geopy lib
<li><var>Second one is mathematical calculation

In [27]:
#did not have time to refactor
def calc_dist1(or_lat, or_lon, dest_lat, dest_lon):
    '''
    DOCSTRING:Function for calculating the distance between two locations
    based on long& lat information in KMs
    '''
    distances=[]
    for x in zip(or_lat, or_lon, dest_lat, dest_lon):
        lat1, lon1, lat2, lon2 =x
        coords_1 = (lat1, lon1)
        coords_2 = (lat2, lon2)
        distance=geopy.distance.distance(coords_1, coords_2).km
        distances.append(distance)
    return distances

In [28]:
#did not have time to refactor
def calc_dist2(or_lat, or_lon, dest_lat, dest_lon):
    '''
    DOCSTRING:Function for calculating the distance between two locations
    based on long& lat information in KMs
    '''
    radius = 6371  # For km
    distances=[]
    for x in zip(or_lat, or_lon, dest_lat, dest_lon):
        lat1, lon1, lat2, lon2 =x
        #if (-90.0 <= lat1 <= 90) and (-90.0 <= lat2 <= 90) and (-180.0 <= lon1 <= 80) and (-180.0 <= lon2 <= 80)
        dlat = math.radians(lat2 - lat1)
        dlon = math.radians(lon2 - lon1)
        a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
                math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
                math.sin(dlon / 2) * math.sin(dlon / 2))
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = radius * c
        distances.append(distance)
    return distances

In [29]:
df['vac_distance1']=calc_dist1(df['origin_lat'], df['origin_lon'], df['dest_lat'], df['dest_lon'])

In [30]:
df['vac_distance2']=calc_dist2(df['origin_lat'], df['origin_lon'], df['dest_lat'], df['dest_lon'])

In [31]:
#Encoding of origins and destinations
X_dum=pd.get_dummies(df[['destination','origin']], dummy_na=True,drop_first=False)
X_dum.drop(['origin_nan','destination_nan'], axis=1, inplace=True)

In [32]:
#Concantanting the dummy variables to the dataframe
df=pd.concat([df, X_dum], axis=1)

In [33]:
#I dont want to lose this version of data so I have created a copy with a different name
df_staging=df

In [34]:
df_staging.to_csv('staging_features.csv')

## User Features Aggregations
In the staging dataframe all features were calculated based on transaction rows. But in the model it is a better idea to utilize user based features since each prediction will be performed for each individual user. 
Thats why I decided to aggregate features on user level.

In [35]:
#To display all columns
pd.options.display.max_columns = 10000
pd.options.display.max_columns = 600
#pd.set_option('display.max_columns', 10000)
#pd.set_option('display.max_rows', 10000)
cols=list(df.columns)
cols

['ts',
 'event_type',
 'user_id',
 'date_from',
 'date_to',
 'origin',
 'destination',
 'num_adults',
 'num_children',
 'origin_lat',
 'origin_lon',
 'dest_lat',
 'dest_lon',
 'vac_duration',
 'book_search_duration',
 'book_days_ago',
 'child_per_adult',
 'booked',
 'searched',
 'is_summer',
 'christmas',
 'vac_distance1',
 'vac_distance2',
 'destination_ABV',
 'destination_ADB',
 'destination_ADD',
 'destination_ADL',
 'destination_AER',
 'destination_AGP',
 'destination_AKL',
 'destination_ALA',
 'destination_ALC',
 'destination_ALG',
 'destination_AMS',
 'destination_ANC',
 'destination_ANK',
 'destination_ARN',
 'destination_ATH',
 'destination_ATL',
 'destination_AUH',
 'destination_AUS',
 'destination_AYT',
 'destination_BAH',
 'destination_BCN',
 'destination_BER',
 'destination_BGY',
 'destination_BHX',
 'destination_BHZ',
 'destination_BJS',
 'destination_BKK',
 'destination_BLR',
 'destination_BNA',
 'destination_BNE',
 'destination_BOG',
 'destination_BOM',
 'destination_BOS

In [36]:
#I have classified columns. Some of them needs to be summed up, for the others I will use mean values
to_sum=[col for col in cols if col.startswith('destination') or col.startswith('origin') 
        or col in ('user_id','booked','searched','is_summer','is_holiday','christmas') ]
to_mean=['user_id','num_adults','num_children','vac_duration','book_search_duration','book_days_ago','child_per_adult','vac_distance1','vac_distance2']

In [37]:
df_sum=df[to_sum]
df_mean=df[to_mean]

In [38]:
df_sum_ag=df_sum.groupby('user_id').agg('sum')

In [39]:
df_sum_ag

,origin_lat,origin_lon,booked,searched,is_summer,christmas,destination_ABV,destination_ADB,destination_ADD,destination_ADL,destination_AER,destination_AGP,destination_AKL,destination_ALA,destination_ALC,destination_ALG,destination_AMS,destination_ANC,destination_ANK,destination_ARN,destination_ATH,destination_ATL,destination_AUH,destination_AUS,destination_AYT,destination_BAH,destination_BCN,destination_BER,destination_BGY,destination_BHX,destination_BHZ,destination_BJS,destination_BKK,destination_BLR,destination_BNA,destination_BNE,destination_BOG,destination_BOM,destination_BOS,destination_BPN,destination_BRU,destination_BSB,destination_BUD,destination_BUE,destination_BUH,destination_BWI,destination_BZE,destination_CAI,destination_CAN,destination_CAS,destination_CCS,destination_CCU,destination_CDG,destination_CEB,destination_CGH,destination_CGK,destination_CGN,destination_CGO,destination_CHC,destination_CHI,destination_CJU,destination_CKG,destination_CLT,destination_CMB,destination_CMN,destination_CNF,destination_CNS,destination_CPH,destination_CPT,destination_CSX,destination_CTS,destination_CTU,destination_CUN,destination_DAC,destination_DAL,destination_DAR,destination_DCA,destination_DEL,destination_DEN,destination_DFW,destination_DLC,destination_DME,destination_DMK,destination_DMM,destination_DOH,destination_DPS,destination_DTT,destination_DTW,destination_DUB,destination_DUR,destination_DUS,destination_DXB,destination_EDI,destination_ESB,destination_EWR,destination_EZE,destination_FAI,destination_FCO,destination_FLL,destination_FRA,destination_FUK,destination_GDL,destination_GIG,destination_GLA,destination_GMP,destination_GRU,destination_GVA,destination_HAK,destination_HAM,destination_HAN,destination_HAV,destination_HEL,destination_HET,destination_HGH,destination_HKG,destination_HKT,destination_HND,destination_HNL,destination_HOU,destination_HYD,destination_IAD,destination_IAH,destination_ICN,destination_IEV,destination_IST,destination_IZM,destination_JED,destination_JFK,destination_JKT,destination_JNB,destination_KHI,destination_KIX,destination_KMG,destination_KNO,destination_KRT,destination_KTM,destination_KUL,destination_KWE,destination_KWI,destination_LAD,destination_LAS,destination_LAX,destination_LED,destination_LGA,destination_LGW,destination_LHR,destination_LIM,destination_LIS,destination_LON,destination_LOS,destination_LPA,destination_LTN,destination_LYS,destination_MAA,destination_MAD,destination_MAN,destination_MCI,destination_MCO,destination_MCT,destination_MDE,destination_MED,destination_MEL,destination_MES,destination_MEX,destination_MHD,destination_MIA,destination_MIL,destination_MKC,destination_MNL,destination_MOW,destination_MRS,destination_MSP,destination_MSY,destination_MTY,destination_MUC,destination_MXP,destination_NAN,destination_NAS,destination_NBO,destination_NCE,destination_NGO,destination_NKG,destination_NOU,destination_NRT,destination_NYC,destination_OAK,destination_OKA,destination_OOL,destination_OPO,destination_ORD,destination_ORL,destination_ORY,destination_OSA,destination_OSL,destination_OTP,destination_OVB,destination_PAR,destination_PDX,destination_PEK,destination_PER,destination_PHL,destination_PHX,destination_PIT,destination_PMI,destination_POA,destination_PPT,destination_PRG,destination_PTY,destination_PVG,destination_RDU,destination_REC,destination_REK,destination_RGN,destination_RIO,destination_ROM,destination_RUH,destination_SAN,destination_SAO,destination_SAW,destination_SCL,destination_SDU,destination_SEA,destination_SEL,destination_SFO,destination_SGN,destination_SHA,destination_SHE,destination_SHJ,destination_SIA,destination_SIN,destination_SIP,destination_SJC,destination_SJO,destination_SJU,destination_SLC,destination_SMF,destination_SPK,destination_SSA,destination_STL,destination_STN,destination_STO,destination_STR,destination_SUB,destination_SVO,destination_SXF,destination_SYD,destination_SYX,destination_SZX,destination_TAO,destination_TCI,destination_THR,destination_TLS,desti

In [40]:
df_mean_ag=df_mean.groupby('user_id').agg('mean')

In [41]:
df_mean_ag

,num_adults,num_children,vac_duration,book_search_duration,book_days_ago,child_per_adult,vac_distance1,vac_distance2
user_id,,,,,,,,
000069,2.0,3.0,16.0,10.0,1187.0,1.5,1548.008799,1548.875108
00019d,4.0,0.0,28.0,139.0,1188.0,0.0,7932.127859,7951.952030
0002e3,1.0,0.0,0.0,6.0,1192.0,0.0,2107.313051,2104.104322
000332,1.5,0.0,29.0,41.0,1189.0,0.0,2731.559030,2727.630372
0004e1,1.0,0.0,6.0,80.0,1188.0,0.0,709.283337,709.657571
...,...,...,...,...,...,...,...,...
fff74a,1.0,0.0,3.0,31.0,1187.0,0.0,1658.375167,1657.705272
fff86d,1.0,0.0,20.0,131.0,1188.0,0.0,18548.368387,18555.994714
fff8fc,2.0,2.0,22.0,106.0,1186.0,1.0,12399.118366,12397.763237


In [42]:
df_user_features=pd.concat([df_mean_ag,df_sum_ag],axis=1)

In [43]:
df_user_features['user_id']=df_user_features.index

In [44]:
# this the aggregated dataframe for users
df_user_features.head()

,num_adults,num_children,vac_duration,book_search_duration,book_days_ago,child_per_adult,vac_distance1,vac_distance2,origin_lat,origin_lon,booked,searched,is_summer,christmas,destination_ABV,destination_ADB,destination_ADD,destination_ADL,destination_AER,destination_AGP,destination_AKL,destination_ALA,destination_ALC,destination_ALG,destination_AMS,destination_ANC,destination_ANK,destination_ARN,destination_ATH,destination_ATL,destination_AUH,destination_AUS,destination_AYT,destination_BAH,destination_BCN,destination_BER,destination_BGY,destination_BHX,destination_BHZ,destination_BJS,destination_BKK,destination_BLR,destination_BNA,destination_BNE,destination_BOG,destination_BOM,destination_BOS,destination_BPN,destination_BRU,destination_BSB,destination_BUD,destination_BUE,destination_BUH,destination_BWI,destination_BZE,destination_CAI,destination_CAN,destination_CAS,destination_CCS,destination_CCU,destination_CDG,destination_CEB,destination_CGH,destination_CGK,destination_CGN,destination_CGO,destination_CHC,destination_CHI,destination_CJU,destination_CKG,destination_CLT,destination_CMB,destination_CMN,destination_CNF,destination_CNS,destination_CPH,destination_CPT,destination_CSX,destination_CTS,destination_CTU,destination_CUN,destination_DAC,destination_DAL,destination_DAR,destination_DCA,destination_DEL,destination_DEN,destination_DFW,destination_DLC,destination_DME,destination_DMK,destination_DMM,destination_DOH,destination_DPS,destination_DTT,destination_DTW,destination_DUB,destination_DUR,destination_DUS,destination_DXB,destination_EDI,destination_ESB,destination_EWR,destination_EZE,destination_FAI,destination_FCO,destination_FLL,destination_FRA,destination_FUK,destination_GDL,destination_GIG,destination_GLA,destination_GMP,destination_GRU,destination_GVA,destination_HAK,destination_HAM,destination_HAN,destination_HAV,destination_HEL,destination_HET,destination_HGH,destination_HKG,destination_HKT,destination_HND,destination_HNL,destination_HOU,destination_HYD,destination_IAD,destination_IAH,destination_ICN,destination_IEV,destination_IST,destination_IZM,destination_JED,destination_JFK,destination_JKT,destination_JNB,destination_KHI,destination_KIX,destination_KMG,destination_KNO,destination_KRT,destination_KTM,destination_KUL,destination_KWE,destination_KWI,destination_LAD,destination_LAS,destination_LAX,destination_LED,destination_LGA,destination_LGW,destination_LHR,destination_LIM,destination_LIS,destination_LON,destination_LOS,destination_LPA,destination_LTN,destination_LYS,destination_MAA,destination_MAD,destination_MAN,destination_MCI,destination_MCO,destination_MCT,destination_MDE,destination_MED,destination_MEL,destination_MES,destination_MEX,destination_MHD,destination_MIA,destination_MIL,destination_MKC,destination_MNL,destination_MOW,destination_MRS,destination_MSP,destination_MSY,destination_MTY,destination_MUC,destination_MXP,destination_NAN,destination_NAS,destination_NBO,destination_NCE,destination_NGO,destination_NKG,destination_NOU,destination_NRT,destination_NYC,destination_OAK,destination_OKA,destination_OOL,destination_OPO,destination_ORD,destination_ORL,destination_ORY,destination_OSA,destination_OSL,destination_OTP,destination_OVB,destination_PAR,destination_PDX,destination_PEK,destination_PER,destination_PHL,destination_PHX,destination_PIT,destination_PMI,destination_POA,destination_PPT,destination_PRG,destination_PTY,destination_PVG,destination_RDU,destination_REC,destination_REK,destination_RGN,destination_RIO,destination_ROM,destination_RUH,destination_SAN,destination_SAO,destination_SAW,destination_SCL,destination_SDU,destination_SEA,destination_SEL,destination_SFO,destination_SGN,destination_SHA,destination_SHE,destination_SHJ,destination_SIA,destination_SIN,destination_SIP,destination_SJC,destination_SJO,destination_SJU,destination_SLC,destination_SMF,destination_SPK,destination_SSA,destination_STL,destination_STN,destination_STO,destination_STR,destination_SUB,destination_SVO,destination_SXF,d

In [45]:
# time to export features so that I can use them in other notebooks
df_user_features.to_csv('user_features.csv')